# ADC exercises

In this exercise, we will implement the closed-shell restricted ADC(1) matrix yourself and diagonalize it to obtain excitation energies. We will use [Gator](sec:gator) to compare the results and obtain both singlet and triplet excitation energies. Furthermore, we will consider the core-valence separation (CVS) approximation for the calculation of core-excited states.

First, we define a small molecule (lithium hydride) and minimal basis set (STO-3G).

```python
import veloxchem as vlx
import gator
import py3Dmol as p3d
import numpy as np
from veloxchem.mointsdriver import MOIntegralsDriver

np.set_printoptions(precision=5, suppress=True)
# Conversion from Hartree to eV for later use
au2ev = vlx.veloxchemlib.hartree_in_ev()
print(au2ev)

lih_xyz="""2

Li  0.000000   0.000000   0.000000
H   0.000000   0.000000   1.000000
"""
lih = vlx.Molecule.from_xyz_string(lih_xyz)

# Method and basis set
method = 'adc1'
basis_set_label = "sto-3g"
basis = vlx.MolecularBasis.read(lih, basis_set_label)
```

## Calculate reference state

```python
# SCF will be run by VeloxChem through Gator
scf = gator.run_scf(lih, basis, conv_thresh=1e-10, verbose=True)
```

**Look at the molecular orbitals**

- Can you assign symmetry labels to the orbitals?
  - $\sigma$ orbitals are rotationally symmetric around the molecular axis (projection of the electron's angular momentum onto the molecular axis $\lambda = 0$)
  - $\pi$ orbitals have one nodal plane along the axis ($\lambda = \pm 1$)

```python
orb_viewer = vlx.OrbitalViewer()
orb_viewer.plot(lih, basis, scf.mol_orbs)
```

## Contruct the ADC(1) matrix

The ADC(1) matrix $\mathbf{M}$ is in a *spin-orbital basis* given by

$$
  M_{ia,jb} = (\epsilon_a - \epsilon_i) \delta_{ij} \delta_{ab} - \langle ja || ib \rangle \, ,
$$

which for a closed-shell restricted HF reference can be written in terms of _spatial_ orbitals as

$$
   M_{ia,jb} = (\epsilon_a - \epsilon_i) \delta_{ij} \delta_{ab} + 2 \langle i j | a b \rangle  - \langle j a | i b \rangle \, ,
$$

where $\epsilon_p$ are HF orbital energies and $\langle p q | r s \rangle$ are two-electron repulsion integrals in physicists' ("1212") notation.

- We look at a very simple example of the LiH molecule in a minimal basis (STO-3G), having two occupied and four virtual orbitals
- The first task is to construct the closed-shell ADC(1) matrix "by hand." For this, we need the HF orbital energies (collected in a diagonal matrix) and the two-electron integrals
- You can also construct the ADC(0) matrix, which consists only of the orbital-energy differences on the diagonal

Extract MO energies and two-electron integrals:

```python
# MO energies in diagonal matrix
mo_energies = np.diag(scf.scf_tensors['E'])

# Two-electron integrals from vlx
moints_drv = MOIntegralsDriver()
oovv = moints_drv.compute_in_mem(lih, basis, scf.mol_orbs, "OOVV")
ovov = moints_drv.compute_in_mem(lih, basis, scf.mol_orbs, "OVOV")
```

Construct matrix:

```python
# Number of occupied and virtual orbitals
nocc = lih.number_of_alpha_electrons()
norb = mo_energies.shape[0]
nvir = norb - nocc
nexc = nocc * nvir # number of excited (singlet or triplet) states in ADC(1)
print("nocc: ", nocc)
print("nvir: ", nvir)

# ADC matrices with zeroes
adc0_mat4d = np.zeros((nocc, nvir, nocc, nvir))
adc1_mat4d = np.zeros((nocc, nvir, nocc, nvir))

# Loop over all indices and fill matrix with corresponding elements
for i in range(nocc):
    for a in range(nvir):
        # Fill the diagonal (orbital-energy differences)
        adc0_mat4d[i,a,i,a] = ...
        adc1_mat4d[i,a,i,a] = ...
        for j in range(nocc):
            for b in range(nvir):
                # Fill the rest (two-electron integrals)
                ...
```

Reshape:

```python
# Reshape the 4D into 2D matrices (and print them if you like)
adc0_mat2d = ...
#print("ADC(0) matrix:\n", adc0_mat2d.shape, "\n", adc0_mat2d)

adc1_mat2d = ...
#print("\nclosed-shell ADC(1) matrix:\n", adc1_mat2d.shape, "\n", adc1_mat2d)
```

And iagonalize the ADC(1) matrix:

```python
adc1_eigvals = np.linalg.eigvalsh(...)
print("Closed-shell ADC(1)/STO-3G eigenvalues:\n", adc1_eigvals)
```

- Now we want to compare the obtained eigenvalues with the results from Gator

## ADC calculation in Gator

```python
adc_results = gator.run_adc(lih, basis, scf, verbose=False, method='adc1',
                            singlets=nexc, tol=1e-5)
```

```python
# We can print a summary of the results
print(adc_results.describe())

# Or plot the absorbtion spectrum
adc_results.plot_spectrum(label='adc(1)')
```

```python
# Compare the two results for the excitation energies
# Gator saves them in the member variable "excitation_energy"
print("Closed-shell ADC(1)/STO-3G eigenvalues:\n", ...)
print("\nADC(1)/STO-3G singlet excitation energies:\n", ...)
```

- You should see that the two results are identical

## Obtain and diagonalize the full ADC(1) matrix

- One can get the full ADC(1) matrix in spin-orbital basis from Gator
- Confirm that it is four times as large as the closed-shell restricted one (having $\alpha\alpha, \alpha\beta, \beta\alpha$, and $\beta\beta$ blocks)
- The next task is to diagonalize it and have a look at the excitation energies and their degeneracies
- Can you assign term symbols (singlet or triplet, $\Sigma$ or $\Pi$ state) to the respective excited states?
    - Hint: the symmetry labels of the orbitals and degeneracies should be able to help you

```python
# This is the full matrix
gator_adc_matrix = adc_results.matrix.to_ndarray()
# Print the matrix and its shape
...
```

```python
# Diagonalize the ADC(1) matrix from Gator to obtain excitation energies and vectors
gator_adc1_eigvals, gator_adc1_eigvecs = np.linalg.eigh(...)

print("Excitation energies from spin-orbital ADC(1)/STO-3G matrix:\n", ...)
```

```python
# With gator, one can also look at the excitation vectors ...
print("ph vector shape:", adc_results.excitation_vector[0].ph.shape)

# ... and have it print out the most important vector amplitudes
print(adc_results.describe_amplitudes())
```

- You can also calculate the triplet states to confirm that your assignments were correct
- Info: setting `states=...` calculates both singlet and triplet states

```python
# Calculate triplet excited states
adc_triplet_results = gator.run_adc(lih, basis, scf, verbose=False, method='adc1',
                            triplets=nexc, tol=1e-5)

# Print the triplet excitation energies and compare 
...
```

- Note that you can calculate 8 triplet states (at most), but they are all triply degenerate, so in total this corresponds to 24 states
- Together with the 8 singlets this gives 32 states, the dimension of the full ADC(1)/STO-3G matrix for LiH

## Core-excited states and core-valence separation (CVS) approximation

- From the above results for the (singlet or triplet) excitation energies, you see that they can be divided into two groups (consisting of 4 excited states each):
  - Those with an excitation energy $<$0.75 Hartree (singlets) or $<$0.5 Hartree (triplets)
  - Those with an excitation energy $>$2 Hartree
- The first group can be considered __valence excitations__ (from the filled bonding orbital), the second group are __core excitations__ (from the Li 1s orbital)
- __Core-excited states__ can also be targeted directly by employing the CVS approximation
- Perform a CVS-ADC(1) calculation with Gator by setting the corresponding method, adjusting the number of states, and setting `core_orbitals=1`
- Compare the excitation energies with the corresponding ones from the previous calculation

```python
# Run CVS-ADC(1) calculation with Gator
...

# Compare the CVS-ADC(1) results with the core-excitation energies from the full (singlet) diagonalization (convert to eV)
...
```

- Repeat the SCF calculation with the NaH molecule (just replace Li by Na in the molecule string)
- Diagonalize the ADC(1) matrix for all singlets (with Gator or your own implementation)
- Compare the core-excitation energies to the results from a corresponding CVS-ADC(1) calculation

```python
# Sodium hydride xyz string and molecule object
...

# Basis set
...

# Run SCF through Gator
...
```

```python
# Number of MOs, occupied and virtual orbitals, and excited states for NaH
...
```

```python
# Calculate all possible singlet excited states with ADC(1)
# How many are there?
...

# Print the excitation energies
...
```

```python
# Calculate all possible core-excited states with CVS-ADC(1)
# How many are there?
...

# Compare the results of the core-excitation energies
...
```

- You should see that the CVS approximation is even better for NaH than for LiH, which is due to the larger energy separation of the core and valence orbitals